In [ ]:
import os
import pprint
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# MODEL = 'gpt-3.5-turbo'
# MODEL = 'mixtral:8x7b'
MODEL = 'llama3'
from langchain_openai.chat_models import ChatOpenAI

# model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
# model.invoke("how many population is it in suzhou?")

In [ ]:
from langchain_community.llms import Ollama
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
if MODEL.startswith('gpt'):
   model =  ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
   embeddings = OpenAIEmbeddings()
else:
  model = Ollama( model=MODEL) 
  embeddings = OllamaEmbeddings(model=MODEL) 
model.invoke("what's the population of Nepal?")  

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
chain = model | parser
chain.invoke('tell me a fun fact of the year 2021')


In [ ]:
from langchain.prompts import PromptTemplate
template = """
 Answer the question based on the context below. If you can't 
answer the question, reply "I have no idea.".
Context: {context}
Question: {question}
"""
prompt = PromptTemplate.from_template(template)
prompt.format(context="Paris is the capital of France", question="What is the capital of France?")

In [ ]:
chain = prompt | model | parser
chain.invoke({ "context":"I am from Pizhou", "question":"how much money do i earn?"})

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("mlschool.pdf")
pages = loader.load_and_split()
pages

In [ ]:
from langchain.prompts import PromptTemplate
template = """
 Answer the question based on the context below. If you can't 
answer the question, reply "I have no idea.".
Context: {context}
Question: {question}
"""
prompt = PromptTemplate.from_template(template)
result = prompt.format(context="Here is some context", question="Here is the question")
print(result)

In [ ]:
chain = prompt | model | parser
chain.input_schema.schema()

In [ ]:
chain.invoke({ "context":"my wife call me superman.", "question":"what does my wife call me?"})

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch
vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [ ]:
retriver = vectorstore.as_retriever()
retriver.invoke('Machine learning')

In [ ]:
from operator import itemgetter
chain =(
{
  "context": itemgetter('question') | retriver,
  "question": itemgetter('question')
}
| prompt | model | parser)
chain.invoke({'question':"what's the purpose of the course?"})

In [ ]:
questions = [
    "What is the purpose of the course?",
    "How many hours of live sessions?",
    "How many coding assignments are there in the program?",
    "Is there a program certificate upon completion?",
    "What programming language will be used in the program?",
    "How much does the program cost?",
]
for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({ 'question': question })}\n")

In [ ]:

for s in chain.stream({'question': "How much does the program cost?"}):
  print(s, end="", flush=True)

In [ ]:
chain.batch([{ "question": question } for question in questions])